# Lesson 4: Aggregate Functions and GROUP BY

**Duration:** 20 minutes  
**Prerequisites:** Complete Lessons 1-3  
**Learning Mode:** Read explanations, then run each SQL query

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
- Use aggregate functions (COUNT, AVG, MAX, MIN, SUM)
- Group data with GROUP BY
- Filter groups with HAVING
- Understand the difference between WHERE and HAVING
- Calculate statistics on your data


## 📚 What are Aggregate Functions?

**Aggregate functions** perform calculations on multiple rows and return a single result. They help you answer questions like:

- How many characters are in the database?
- What's the average height?
- Who is the tallest character?
- How many characters of each species are there?

### The Five Main Aggregate Functions

| Function | Purpose | Example Result |
|----------|---------|----------------|
| `COUNT()` | Count rows | 11 |
| `AVG()` | Calculate average | 165.7 |
| `MAX()` | Find maximum value | 228 |
| `MIN()` | Find minimum value | 66 |
| `SUM()` | Add up values | 1823 |


## 🛠️ Setup: Connect to Database

**Run the next 3 cells:**


In [27]:
# Load SQL magic extension
%load_ext sql

# Fix prettytable compatibility issue
import prettytable
try:
    # Try to access DEFAULT to see if it exists
    _ = prettytable.DEFAULT
except AttributeError:
    # If it doesn't exist, add it using SINGLE_BORDER
    from prettytable import SINGLE_BORDER
    prettytable.DEFAULT = SINGLE_BORDER

# Configure SQL magic settings
%config SqlMagic.autopandas = False
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [28]:
%sql sqlite:///starwars.db

In [29]:
%%sql
-- Reference: See all data with species breakdown
SELECT name, species, height FROM characters ORDER BY species, height;


name,species,height
R2-D2,Droid,96
Leia Organa,Human,150
Padmé Amidala,Human,165
Luke Skywalker,Human,172
Han Solo,Human,180
Obi-Wan Kenobi,Human,182
Mace Windu,Human,188
Darth Vader,Human,202
Ahsoka Tano,Togruta,170
Chewbacca,Wookiee,228


## 📚 Data Reference for Aggregates

**Character heights (useful for this lesson):**
- Yoda: 66cm
- R2-D2: 96cm  
- Princess Leia: 150cm
- Luke Skywalker: 172cm
- Darth Vader: 202cm
- Chewbacca: 228cm

**Species counts:**
- Human: 3 characters
- Droid: 1 character
- Wookiee: 1 character
- Yoda's species: 1 character

💡 **Use aggregates to calculate:** COUNT(*), AVG(height), MAX(height), MIN(height), SUM(height)


## 🔢 Part 1: Basic Aggregate Functions

### Query 1: Count All Characters

**Result:** Total number of rows (e.g., 11)


In [30]:
%%sql
-- Query 1: Count how many characters are in the table
SELECT COUNT(*) from characters

COUNT(*)
11


### Query 2: Count Non-NULL Values

**Note:** `COUNT(column)` ignores NULL values, but `COUNT(*)` counts all rows.


In [31]:
%%sql
-- Query 2: Count characters who have a height recorded
SELECT count(*) from characters where height is not null

count(*)
11


### Query 3: Find Maximum Height

**Result:** Should show 228 (Chewbacca's height)


In [32]:
%%sql
-- Query 3: Find the tallest character's height
SELECT MAX(height) from characters

MAX(height)
228


### Query 4: Find Minimum Height

**Result:** Should show 66 (Yoda's height)


In [33]:
%%sql
-- Query 4: Find the shortest character's height
SELECT MIN(height) from characters

MIN(height)
66


### Query 5: Calculate Average Height

**Result:** Approximately 158 cm (average of all characters with height data)


In [34]:
%%sql
-- Query 5: Calculate average height
select avg(height) from characters

avg(height)
163.54545454545453


### Query 6: Sum All Heights

Add up all character heights together:


In [35]:
%%sql
-- Query 6: Sum all heights
select sum(height) from characters

sum(height)
1799


### Query 7: Use Multiple Aggregates Together

**Explanation:** `AS` creates an alias (friendly name) for the result column.


In [36]:
%%sql
-- Query 7: Get stats in one query with column aliases
select name as n, height as h from characters

n,h
Luke Skywalker,172
Leia Organa,150
Han Solo,180
Chewbacca,228
Obi-Wan Kenobi,182
Darth Vader,202
Yoda,66
R2-D2,96
Padmé Amidala,165
Mace Windu,188


## 📊 Part 2: GROUP BY Clause

GROUP BY groups rows that have the same values and performs aggregate functions on each group.

### GROUP BY Syntax

```sql
SELECT column, AGGREGATE_FUNCTION(column)
FROM table_name
GROUP BY column;
```

### Query 8: Count Characters by Species

**Result:** Shows each species and how many characters belong to it.


In [37]:
%%sql
-- Query 8: Count characters by species
select 
    species, 
    count(*) as species_count
from characters
group by species
order by species_count desc;

species,species_count
Human,7
Yoda's species,1
Wookiee,1
Togruta,1
Droid,1


### Query 9: Average Height by Species

**Note:** `WHERE height IS NOT NULL` filters out characters without height data before grouping.


In [38]:
%%sql
-- Query 9: Average height by species
select 
    species, 
    avg(height) as average_height
from characters
where height is not null
group by species
order by average_height desc;

species,average_height
Wookiee,228.0
Human,177.0
Togruta,170.0
Droid,96.0
Yoda's species,66.0


### Query 10: Count Characters by Homeworld

**Explanation:** Groups by homeworld, counts each group, then sorts by count (highest first).


In [39]:
%%sql
-- Query 10: Count and average height by species, sorted by count
select 
    species, 
    avg(height) as average_height, 
    count(*) as species_count
from characters
where height is not null
group by species
order by species_count desc;

species,average_height,species_count
Human,177.0,7
Yoda's species,66.0,1
Wookiee,228.0,1
Togruta,170.0,1
Droid,96.0,1


## 🏛️ Part 3: Add Affiliation Data

Before the next examples, let's add affiliation data to track which organisations characters belong to.

**Run the next 2 cells to add and populate the affiliation column:**


In [40]:
%%sql
-- Query 11: Filter BEFORE grouping - only characters taller than 100cm
select species, height 
from characters
where height > 100
group by species
order by height desc;

species,height
Wookiee,228
Human,172
Togruta,170


In [41]:
%%sql
-- Query 12: Filter AFTER grouping - species with 2+ characters
select 
    species,
    count(*) as species_count
from characters
group by species
having species_count >= 2
order by species_count desc;

species,species_count
Human,7


### Verify Affiliation Data


In [42]:
%%sql
-- Query 13: Count characters by homeworld
select 
    homeworld,
    count(*) as char_count
from characters
group by homeworld
order by char_count desc;

homeworld,char_count
Tatooine,2
Naboo,2
Unknown,1
Stewjon,1
Shili,1
Kashyyyk,1
Haruun Kal,1
Corellia,1
Alderaan,1


### Query 11: Count by Affiliation


In [43]:
%%sql
-- Query 14: Combining WHERE, GROUP BY, HAVING, and ORDER BY
select * from characters

id,name,species,homeworld,height
49,Luke Skywalker,Human,Tatooine,172
50,Leia Organa,Human,Alderaan,150
51,Han Solo,Human,Corellia,180
52,Chewbacca,Wookiee,Kashyyyk,228
53,Obi-Wan Kenobi,Human,Stewjon,182
54,Darth Vader,Human,Tatooine,202
55,Yoda,Yoda's species,Unknown,66
56,R2-D2,Droid,Naboo,96
57,Padmé Amidala,Human,Naboo,165
58,Mace Windu,Human,Haruun Kal,188


## 🎯 Part 4: HAVING Clause

`HAVING` filters groups (like WHERE filters rows). Use HAVING with GROUP BY to filter aggregated results.

### WHERE vs HAVING

| Clause | Filters | Used With | Example |
|--------|---------|-----------|---------|
| `WHERE` | Individual rows | Before grouping | `WHERE species = 'Human'` |
| `HAVING` | Groups | After grouping | `HAVING COUNT(*) > 2` |

### HAVING Syntax

```sql
SELECT column, AGGREGATE_FUNCTION(column)
FROM table_name
GROUP BY column
HAVING condition;
```

### Query 12: Find Species with Multiple Characters

**Result:** Only shows species that have 2+ characters.


In [44]:
%%sql
-- Example 1: Species with at least 2 members and average height > 170cm
select
    species,
    count(*) as species_count,
    avg(height) as avg_height
from characters
group by species
having species_count >= 2 and avg_height > 170

species,species_count,avg_height
Human,7,177.0


### Query 13: Combine WHERE and HAVING

**Explanation:**
1. `WHERE species = 'Human'` - Filter to humans first
2. `GROUP BY homeworld` - Group remaining rows by planet
3. `HAVING COUNT(*) >= 2` - Only show groups with 2+ characters


In [45]:
%%sql
-- Example 2: Homeworlds with at least 2 human characters
select 
    homeworld, 
    count(*) as species_count 
from characters 
where species = 'Human' 
group by homeworld 
having species_count >= 2

homeworld,species_count
Tatooine,2


## �� Part 5: COUNT DISTINCT

`COUNT(DISTINCT column)` counts unique values, ignoring duplicates.

### Query 14: Count Unique Species


In [46]:
%%sql
-- Exercise 1: Find maximum and minimum height
select
    species,
    max(height) as max_height,
    min(height) as min_height
from characters
where height is not null
group by species
order by max_height desc;

species,max_height,min_height
Wookiee,228,228
Human,202,150
Togruta,170,170
Droid,96,96
Yoda's species,66,66


### Query 15: Count Unique Homeworlds


In [47]:
%%sql
-- Exercise 2: Count how many droids
select
    count(*) as clanker_count
from characters
where species = 'Droid'

clanker_count
1


## 🎓 Practice Exercises

Now it's your turn! Complete these queries:


### Exercise 1: Find Total Height


In [48]:
%%sql
-- Exercise 3: Average height by homeworld


[]

### Exercise 2: Group and Count


In [49]:
%%sql
-- Exercise 4: Species with average height greater than 180cm


[]

### Exercise 3: Average with Grouping


In [50]:
%%sql
-- Exercise 5: Total height of all humans


[]

### Exercise 4: Using HAVING


In [51]:
%%sql
-- Challenge Problem


[]

## 🎯 Challenge Problem

**Task:** Find which affiliation has the tallest average height, but only include affiliations with 2 or more members. Show the affiliation name, average height (rounded to 2 decimals), and member count, sorted by average height (tallest first).

**Requirements:**
- Use AVG(), COUNT()
- Use GROUP BY
- Use HAVING to filter groups
- Use ORDER BY
- Use ROUND(AVG(height), 2)


In [52]:
%%sql
-- Challenge Problem Solution
SELECT 
    affiliation,
    COUNT(*) AS member_count,
    ROUND(AVG(height), 2) AS avg_height
FROM characters
WHERE affiliation IS NOT NULL AND height IS NOT NULL
GROUP BY affiliation
HAVING COUNT(*) >= 2
ORDER BY avg_height DESC;


(sqlite3.OperationalError) no such column: affiliation
[SQL: -- Challenge Problem Solution
SELECT 
    affiliation,
    COUNT(*) AS member_count,
    ROUND(AVG(height), 2) AS avg_height
FROM characters
WHERE affiliation IS NOT NULL AND height IS NOT NULL
GROUP BY affiliation
HAVING COUNT(*) >= 2
ORDER BY avg_height DESC;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## 🐛 Common Errors & Troubleshooting

### Error: "misuse of aggregate function"

**Problem:** Using aggregate function without GROUP BY when other columns are selected.

**Wrong:**
```sql
SELECT species, COUNT(*)
FROM characters;
```

**Correct:**
```sql
SELECT species, COUNT(*)
FROM characters
GROUP BY species;
```

**Rule:** If you SELECT a column and an aggregate, you must GROUP BY that column.

### WHERE vs HAVING Confusion

**Wrong:**
```sql
-- Can't use aggregate in WHERE
SELECT species, COUNT(*) 
FROM characters 
WHERE COUNT(*) > 2 
GROUP BY species;
```

**Correct:**
```sql
-- Use HAVING for aggregate conditions
SELECT species, COUNT(*) 
FROM characters 
GROUP BY species
HAVING COUNT(*) > 2;
```

### NULL Values in Aggregates

**Important:** 
- `COUNT(*)` counts all rows (including NULLs)
- `COUNT(column)` counts non-NULL values
- `AVG()`, `SUM()`, `MAX()`, `MIN()` ignore NULLs

### Order of Clauses

**Correct order:**
```sql
SELECT columns
FROM table
WHERE condition       -- Filter rows first
GROUP BY column       -- Then group
HAVING condition      -- Filter groups
ORDER BY column       -- Finally sort
LIMIT number;
```


## ✅ Checkpoint & Summary

### What You've Learnt

- ✅ Use COUNT(), AVG(), MAX(), MIN(), SUM()
- ✅ Group data with GROUP BY
- ✅ Filter groups with HAVING
- ✅ Understand WHERE (rows) vs HAVING (groups)
- ✅ Use COUNT(DISTINCT) for unique values
- ✅ Combine aggregates with ORDER BY and LIMIT
- ✅ Write column aliases with AS

### Key SQL Commands

| Command | Purpose | Example |
|---------|---------|---------|
| `COUNT()` | Count rows | `SELECT COUNT(*) FROM characters` |
| `AVG()` | Calculate average | `SELECT AVG(height) FROM characters` |
| `MAX()` | Find maximum | `SELECT MAX(height) FROM characters` |
| `MIN()` | Find minimum | `SELECT MIN(height) FROM characters` |
| `SUM()` | Add values | `SELECT SUM(height) FROM characters` |
| `GROUP BY` | Group rows | `GROUP BY species` |
| `HAVING` | Filter groups | `HAVING COUNT(*) > 2` |
| `AS` | Column alias | `COUNT(*) AS total` |
| `DISTINCT` | Unique values | `COUNT(DISTINCT species)` |
| `ROUND()` | Round numbers | `ROUND(AVG(height), 2)` |

## 🎉 Excellent Work!

You can now perform calculations and analyse your data! In the next lesson, you'll learn about database relationships and creating multiple related tables.

**Ready to continue?** Open `lesson5_schema.ipynb`

---

## 💾 Git Commands (for reference)

```bash
git status
git add lessons/lesson4_aggregates.ipynb
git commit -m "Completed Lesson 4: Aggregate functions and GROUP BY"
git push
```
